## -------------------------------------- **RR Final Project**  ------------------------------------------
#### DATA SELECTION AND PREPARATION
#### EPL 25 YEARS DATA (2000-2025) FOR ANALYSIS

## ----------------------------------------------------------------------------------------------------------

### Load and Merge All EPL Excel Files

In [ ]:
import pandas as pd
import os

# Path to your data folder
folder_path = r'C:\Users\User\Desktop\Semester 4\Reproducible Research (Online)\RR_project\DATA'

# USE YOUR OWN PATH TO DOWNLOAD SORUCE FILE INTO PYTHON
# Marco -
# Akshima - 
# Ibrahim - 



# Initialize empty list to store dataframes
all_dfs = []

# Loop through season numbers from 00-01 to 24-25
for i in range(25):
    season_str = f"{i:02d}-{i+1:02d}"  # e.g., "00-01", "01-02"
    file_name = f"E0_{season_str}.xlsx"
    file_path = os.path.join(folder_path, file_name)

    if os.path.exists(file_path):
        df = pd.read_excel(file_path)
        df['Season'] = season_str  # Optional: add a season column
        all_dfs.append(df)
    else:
        print(f"File not found: {file_name}")

# Concatenate all dataframes into one
final_df = pd.concat(all_dfs, ignore_index=True)

# Check the result
print("Combined shape:", final_df.shape)
print(final_df.head())


Combined shape: (9440, 188)
  Div      Date  HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  ...  \
0  E0  19/08/00  Charlton       Man City     4     0   H     2     0   H  ...   
1  E0  19/08/00   Chelsea       West Ham     4     2   H     1     0   H  ...   
2  E0  19/08/00  Coventry  Middlesbrough     1     3   A     1     1   D  ...   
3  E0  19/08/00     Derby    Southampton     2     2   D     1     2   A  ...   
4  E0  19/08/00     Leeds        Everton     2     0   H     2     0   H  ...   

   1XBCH 1XBCD  1XBCA  BFECH  BFECD  BFECA  BFEC>2.5  BFEC<2.5  BFECAHH  \
0    NaN   NaN    NaN    NaN    NaN    NaN       NaN       NaN      NaN   
1    NaN   NaN    NaN    NaN    NaN    NaN       NaN       NaN      NaN   
2    NaN   NaN    NaN    NaN    NaN    NaN       NaN       NaN      NaN   
3    NaN   NaN    NaN    NaN    NaN    NaN       NaN       NaN      NaN   
4    NaN   NaN    NaN    NaN    NaN    NaN       NaN       NaN      NaN   

   BFECAHA  
0      NaN  
1      N

#### Find Common Columns Across All Files

In [13]:
# Store column sets
columns_list = []

# Loop through all 25 files
for i in range(25):
    season_str = f"{i:02d}-{i+1:02d}"
    file_name = f"E0_{season_str}.xlsx"
    file_path = os.path.join(folder_path, file_name)

    try:
        df = pd.read_excel(file_path, nrows=1)  # Read only the first row for speed
        columns_list.append(set(df.columns))
    except Exception as e:
        print(f"Error reading {file_name}: {e}")
        
# Find common columns
common_columns = set.intersection(*columns_list)

# Output
print(f"✅ Common variables in all 25 datasets ({len(common_columns)} total):")
for col in sorted(common_columns):
    print("-", col)

✅ Common variables in all 25 datasets (26 total):
- AC
- AF
- AR
- AS
- AST
- AY
- AwayTeam
- Date
- Div
- FTAG
- FTHG
- FTR
- HC
- HF
- HR
- HS
- HST
- HTAG
- HTHG
- HTR
- HY
- HomeTeam
- Referee
- WHA
- WHD
- WHH


#### Filter by Common Variables and Merge All 25 EPL Files and correct Date var

In [14]:
import pandas as pd
import os
import re

folder_path = r'C:\Users\User\Desktop\Semester 4\Reproducible Research (Online)\RR_project\DATA'

common_columns = [
    'AC', 'AF', 'AR', 'AS', 'AST', 'AY', 'AwayTeam', 'Date', 'Div', 'FTAG',
    'FTHG', 'FTR', 'HC', 'HF', 'HR', 'HS', 'HST', 'HTAG', 'HTHG', 'HTR',
    'HY', 'HomeTeam', 'Referee', 'WHA', 'WHD', 'WHH'
]

filtered_dfs = []

# Re-import with raw Date as string
for i in range(25):
    season_str = f"{i:02d}-{i+1:02d}"
    file_name = f"E0_{season_str}.xlsx"
    file_path = os.path.join(folder_path, file_name)

    try:
        # Force Date column to be read as string
        df = pd.read_excel(file_path, dtype={'Date': str}, usecols=common_columns)

        # Fix short year format
        def fix_date_str(date_str):
            if isinstance(date_str, str) and re.match(r'^\d{1,2}/\d{1,2}/\d{2}$', date_str):
                parts = date_str.split('/')
                year = int(parts[2])
                if year < 30:
                    parts[2] = f"20{year:02d}"
                    return '/'.join(parts)
            return date_str

        df['Date'] = df['Date'].apply(fix_date_str)
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

        df['Season'] = season_str
        filtered_dfs.append(df)

    except Exception as e:
        print(f"❌ Error in file {file_name}: {e}")

# Combine all
final_filtered_df = pd.concat(filtered_dfs, ignore_index=True)

# Final validation
invalid_dates = final_filtered_df[final_filtered_df['Date'] < '2000-01-01']
print(f"❌ Number of rows still before 2000: {invalid_dates.shape[0]}")
print("✅ Final valid Date Range:", final_filtered_df['Date'].min(), "to", final_filtered_df['Date'].max())


C:\Users\User\AppData\Local\Temp\ipykernel_21336\1160168750.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_21336\1160168750.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_21336\1160168750.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
C:\Users\User\AppData\Loc

❌ Number of rows still before 2000: 0
✅ Final valid Date Range: 2000-08-19 00:00:00 to 2025-04-16 00:00:00


In [15]:
# Fix the Date column (handles multiple formats, day-first style)
final_filtered_df['Date'] = pd.to_datetime(final_filtered_df['Date'], dayfirst=True, errors='coerce')

# Optional: Drop rows where the Date couldn't be parsed
final_filtered_df = final_filtered_df.dropna(subset=['Date'])

# Check if it worked
print("✅ Date column fixed. Range:", final_filtered_df['Date'].min(), "to", final_filtered_df['Date'].max())
print(final_filtered_df[['Date', 'HomeTeam', 'AwayTeam']].head())


✅ Date column fixed. Range: 2000-08-19 00:00:00 to 2025-04-16 00:00:00
        Date  HomeTeam       AwayTeam
0 2000-08-19  Charlton       Man City
1 2000-08-19   Chelsea       West Ham
2 2000-08-19  Coventry  Middlesbrough
3 2000-08-19     Derby    Southampton
4 2000-08-19     Leeds        Everton


In [16]:
# Find rows with dates before 2000
before_2000 = final_filtered_df[final_filtered_df['Date'] < '2000-01-01']
print(f"❌ Rows with date before 2000: {before_2000.shape[0]}")
print(before_2000[['Date', 'HomeTeam', 'AwayTeam']].head())

# Find rows with dates after 2025
after_2025 = final_filtered_df[final_filtered_df['Date'] > '2025-01-01']
print(f"❌ Rows with date after 2025: {after_2025.shape[0]}")
print(after_2025[['Date', 'HomeTeam', 'AwayTeam']].head())


❌ Rows with date before 2000: 0
Empty DataFrame
Columns: [Date, HomeTeam, AwayTeam]
Index: []
❌ Rows with date after 2025: 75
           Date       HomeTeam     AwayTeam
9319 2025-01-14      Brentford     Man City
9320 2025-01-14        Chelsea  Bournemouth
9321 2025-01-14       West Ham       Fulham
9322 2025-01-14  Nott'm Forest    Liverpool
9323 2025-01-15        Everton  Aston Villa


In [17]:
# Show the first few rows of the first filtered dataset
filtered_dfs[0].head()


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HF,AF,HY,AY,HR,AR,WHH,WHD,WHA,Season
0,E0,2000-08-19,Charlton,Man City,4,0,H,2,0,H,...,13,12,1,2,0,0,2.10,3.2,3.10,00-01
1,E0,2000-08-19,Chelsea,West Ham,4,2,H,1,0,H,...,19,14,1,2,0,0,1.44,3.6,6.50,00-01
2,E0,2000-08-19,Coventry,Middlesbrough,1,3,A,1,1,D,...,15,21,5,3,1,0,2.30,3.2,2.62,00-01
3,E0,2000-08-19,Derby,Southampton,2,2,D,1,2,A,...,11,13,1,1,0,0,2.00,3.2,3.20,00-01
4,E0,2000-08-19,Leeds,Everton,2,0,H,2,0,H,...,21,20,1,3,0,0,1.61,3.5,4.50,00-01


In [18]:
# Combine all datasets into one
final_df = pd.concat(filtered_dfs, ignore_index=True)

# Sort by Date in ascending order
final_df = final_df.sort_values(by='Date', ascending=True).reset_index(drop=True)

# Check result
print("✅ Combined and sorted shape:", final_df.shape)
print(final_df[['Date', 'HomeTeam', 'AwayTeam']].head())


✅ Combined and sorted shape: (9440, 27)
        Date  HomeTeam       AwayTeam
0 2000-08-19  Charlton       Man City
1 2000-08-19   Chelsea       West Ham
2 2000-08-19  Coventry  Middlesbrough
3 2000-08-19     Derby    Southampton
4 2000-08-19     Leeds        Everton


#### **Final Data View**

In [19]:
final_df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HF,AF,HY,AY,HR,AR,WHH,WHD,WHA,Season
0,E0,2000-08-19,Charlton,Man City,4,0,H,2,0,H,...,13,12,1,2,0,0,2.10,3.2,3.10,00-01
1,E0,2000-08-19,Chelsea,West Ham,4,2,H,1,0,H,...,19,14,1,2,0,0,1.44,3.6,6.50,00-01
2,E0,2000-08-19,Coventry,Middlesbrough,1,3,A,1,1,D,...,15,21,5,3,1,0,2.30,3.2,2.62,00-01
3,E0,2000-08-19,Derby,Southampton,2,2,D,1,2,A,...,11,13,1,1,0,0,2.00,3.2,3.20,00-01
4,E0,2000-08-19,Leeds,Everton,2,0,H,2,0,H,...,21,20,1,3,0,0,1.61,3.5,4.50,00-01


In [20]:
final_df.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF',
       'AF', 'HY', 'AY', 'HR', 'AR', 'WHH', 'WHD', 'WHA', 'Season'],
      dtype='object')

In [22]:
# Number of rows and columns
num_rows, num_cols = final_df.shape

print(f"📊 Number of rows: {num_rows}")
print(f"📊 Number of columns: {num_cols}")


📊 Number of rows: 9440
📊 Number of columns: 27


#### **Download the Final Data**

In [23]:
# Save to CSV (no index column)
final_df.to_csv("EPL_Cleaned_2000_to_2025.csv", index=False)

print("✅ File saved as 'EPL_Cleaned_2000_to_2025.csv'")


✅ File saved as 'EPL_Cleaned_2000_to_2025.csv'


In [24]:
# Define full path to Desktop
desktop_path = r'C:\Users\User\Desktop\EPL_Cleaned_2000_to_2025.csv'

# Save the CSV file there
final_df.to_csv(desktop_path, index=False)

print("✅ File successfully saved to your Desktop!")


✅ File successfully saved to your Desktop!
